In [1]:
import hashlib
import json
import time  # Import the time module to work with timestamps and simulate network delays
import numpy as np
import random

In [2]:
class Blockchain:
    def __init__(self):
        # Initialize the blockchain with an empty list for the chain and pending transactions
        self.chain = []
        self.pending_transactions = []  # Store transactions that have not yet been added to a block
        # Create the genesis block (the first block in the chain) with a predefined previous hash
        self.create_block(previous_hash='1', data='Genesis Block', method='sha256')


    def create_block(self, data, previous_hash, method):
        # Create a new block with given data, the hash of the previous block, and a specified hashing method
        block = {
            'index': len(self.chain) + 1,  # Block number in the chain
            'timestamp': time.time(),  # Current timestamp
            'data': data,  # Data to be stored in the block
            'previous_hash': previous_hash,  # Hash of the previous block to maintain the chain's integrity
        }
        block['hash'] = self.hash(block, method)  # Generate the current block's hash
        self.chain.append(block)  # Add the new block to the chain
        self.send_to_central_server(block)  # Simulate sending the block to a centralized server
        return block

    def hash(self, block, method):
        # Generate a hash of the block based on the specified method (SHA-256 or simulated homomorphic encryption)
        encoded_block = json.dumps(block, sort_keys=True).encode()  # Convert the block into a string and encode it
        if method == 'sha256':
            return hashlib.sha256(encoded_block).hexdigest()  # Use SHA-256 hashing
        elif method == 'he':  # Placeholder for homomorphic encryption
            return "he_" + hashlib.sha256(encoded_block).hexdigest()  # Simulate HE by prefixing SHA-256 hash
        else:
            raise ValueError("Unsupported hashing method")  # Raise an error if an unknown method is specified

    def add_block(self, matrix1, matrix2):
        # Add a block to the chain with the result of multiplying two matrices
        previous_block = self.chain[-1]  # Get the last block in the chain
        result_matrix = np.dot(matrix1, matrix2).tolist()  # Perform matrix multiplication and convert the result to a list
        start_time = time.time()  # Record the start time for adding the block
        method = 'sha256' if (len(self.chain) + 1) % 2 == 0 else 'he'  # Alternate between SHA-256 and HE for demonstration
        block = self.create_block(data=result_matrix, previous_hash=previous_block['hash'], method=method)
        end_time = time.time()  # Record the end time for adding the block
        return block, end_time - start_time  # Return the new block and the time taken to add it

    @staticmethod
    def send_to_central_server(block):
        # Simulate sending a block to a centralized server
        print(f"Sending block {block['index']} to the central server with hash {block['hash'][:6]}...")
        time.sleep(1)  # Simulate a network delay
        print(f"Block {block['index']} received by the central server.")
    
    def validate_chain(self):
        # Validate the integrity of the entire blockchain
        for i in range(1, len(self.chain)):
            current_block = self.chain[i]  # Current block being validated
            previous_block = self.chain[i-1]  # Previous block in the chain
            # Check if the previous_hash recorded in the current block matches the hash of the previous block
            if current_block['previous_hash'] != self.hash(previous_block, previous_block.get('method', 'sha256')):
                return f"Chain is altered at block index {i}"  # Report alteration if there's a mismatch
            # Verify the current block's hash is accurate based on its content and method
            if current_block['hash'] != self.hash(current_block, current_block.get('method', 'sha256')):
                return f"Chain is altered at block index {i}"
        return "The chain is valid."  # Return a message indicating the chain is valid if no alterations are found

In [3]:
# Example usage
blockchain = Blockchain()

Sending block 1 to the central server with hash c95714...
Block 1 received by the central server.


In [4]:
# Define two 2x2 matrices
matrix1 = np.array([[1, 2], [3, 4]])
matrix2 = np.array([[5, 6], [7, 8]])

In [5]:
# Add blocks with matrix multiplication results
block1, time1 = blockchain.add_block(matrix1, matrix2)
print(f"Block 1 added in {time1} seconds")
print(json.dumps(block1, indent=4, sort_keys=True))

Sending block 2 to the central server with hash 840d16...
Block 2 received by the central server.
Block 1 added in 1.0007977485656738 seconds
{
    "data": [
        [
            19,
            22
        ],
        [
            43,
            50
        ]
    ],
    "hash": "840d1615f34ffa49707c7b76394f16cff171995ff08f81609753e35661102c77",
    "index": 2,
    "previous_hash": "c957145fc147f5a90776b8e7703d9bf3a75f92c9030906bc85fdd017af077480",
    "timestamp": 1709757559.9355433
}


In [6]:
block2, time2 = blockchain.add_block(matrix1, matrix2)
print(f"Block 2 added in {time2} seconds")
print(json.dumps(block2, indent=4, sort_keys=True))

Sending block 3 to the central server with hash he_0db...
Block 3 received by the central server.
Block 2 added in 1.000504970550537 seconds
{
    "data": [
        [
            19,
            22
        ],
        [
            43,
            50
        ]
    ],
    "hash": "he_0db3251654202f884888ce95d5f7f63a408093bbcf8aff61c6c4c0fa1a644768",
    "index": 3,
    "previous_hash": "840d1615f34ffa49707c7b76394f16cff171995ff08f81609753e35661102c77",
    "timestamp": 1709757563.7672582
}


In [7]:
# Validate the blockchain before any tampering
print(blockchain.validate_chain())

Chain is altered at block index 1


In [8]:
# Simulate tampering
blockchain.chain[1]['data'] = "Tampered Data"

In [9]:
# Validate the blockchain after tampering
print(blockchain.validate_chain())

Chain is altered at block index 1
